In [26]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [124]:
import sys
import caffe
import numpy as np
from caffe import layers as L
from caffe import params as P
from caffe import to_proto
import google.protobuf as pb
from caffe.proto import caffe_pb2

#### Data layer

In [50]:
def data_layer(name, params=None):

    layer = caffe_pb2.LayerParameter()

    layer.name = 'data'

    layer.type = 'Data'

    layer.top.extend(['data'])

    layer.data_param.batch_size = 1

    #layer.data_param.crop_size = 3

#### BN layer

In [70]:
def gen_bn_layer(name, bottom, top):

    bn_layer = caffe_pb2.LayerParameter()

    bn_layer.name = name + '_bn'

    bn_layer.type = 'BatchNorm'

    bn_layer.bottom.extend([bottom])

    bn_layer.top.extend([top])

    bn_layer.batch_norm_param.eps = 1e-5

    bn_layer.batch_norm_param.use_global_stats = True
    
    return bn_layer

#### Scale layer

In [81]:
def gen_scale_layer(name, bottom, top):

    scale_layer = caffe_pb2.LayerParameter()

    scale_layer.name = name + '_scale'

    scale_layer.type = 'Scale'

    scale_layer.bottom.extend([bottom])

    scale_layer.top.extend([top])

    scale_layer.scale_param.filler.value = 1

    scale_layer.scale_param.bias_term = True

    scale_layer.scale_param.bias_filler.value = 0
    
    return scale_layer

#### Generate layers

In [85]:
bn_layer = gen_bn_layer(name='data', bottom='data', top='out')
scale_layer = gen_scale_layer(name='data', bottom='out', top='out')

In [86]:
layers = []
layers.append(bn_layer)
layers.append(scale_layer)

#### Define model

In [88]:
model = caffe_pb2.NetParameter()

model.name = 'test_bn_net'

model.layer.extend(layers)

model.input.append('data')

model.input_dim.append(1)

model.input_dim.append(1)

model.input_dim.append(3)

model.input_dim.append(3)

#### Save model to prototxt

In [92]:
with open('deploy_test_bn_net.prototxt', 'w') as f:
    f.write(pb.text_format.MessageToString(model))

#### Load model and bind data

In [120]:
net = caffe.Net('deploy_test_bn_net.prototxt', caffe.TEST)
net.blobs['data'].data[...] = np.ones(shape=(1,1,3,3))

In [121]:
net.params['data_bn'][0].data[...] = np.array([0])
net.params['data_bn'][1].data[...] = np.array([1])
net.params['data_bn'][2].data[...] = np.array([1])
net.params['data_scale'][0].data[...] = np.array([2])
net.params['data_scale'][1].data[...] = np.array([1])

In [122]:
print 'mean:',net.params['data_bn'][0].data
print 'var:', net.params['data_bn'][1].data
print 'scale:',net.params['data_bn'][2].data
print 'gamma:',net.params['data_scale'][0].data
print 'beta:',net.params['data_scale'][1].data

mean: [0.]
var: [1.]
scale: [1.]
gamma: [2.]
beta: [1.]


In [123]:
net.forward()

{'out': array([[[[2.99999, 2.99999, 2.99999],
          [2.99999, 2.99999, 2.99999],
          [2.99999, 2.99999, 2.99999]]]], dtype=float32)}